In [51]:
import pandas as pd

In [52]:

columns = ["symboling", "normalized-losses", "make", "fuel-type", "aspiration", "num-of-doors", "body-style", "drive-wheels", "engine-location", "wheel-base",
    "length", "width", "height", "curb-weight", "engine-type", "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower",
    "peak-rpm", "city-mpg", "highway-mpg", "price"]

data = pd.read_csv("../data/imports-85.data", names=columns, na_values="?")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  164 non-null    float64
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       203 non-null    object 
 6   body-style         205 non-null    object 
 7   drive-wheels       205 non-null    object 
 8   engine-location    205 non-null    object 
 9   wheel-base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb-weight        205 non-null    int64  
 14  engine-type        205 non-null    object 
 15  num-of-cylinders   205 non-null    object 
 16  engine-size        205 non

Missing data found! removing rows with missing data!

In [53]:
dataDropped = data.dropna().reset_index()
dataDropped.info()

print(dataDropped)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              159 non-null    int64  
 1   symboling          159 non-null    int64  
 2   normalized-losses  159 non-null    float64
 3   make               159 non-null    object 
 4   fuel-type          159 non-null    object 
 5   aspiration         159 non-null    object 
 6   num-of-doors       159 non-null    object 
 7   body-style         159 non-null    object 
 8   drive-wheels       159 non-null    object 
 9   engine-location    159 non-null    object 
 10  wheel-base         159 non-null    float64
 11  length             159 non-null    float64
 12  width              159 non-null    float64
 13  height             159 non-null    float64
 14  curb-weight        159 non-null    int64  
 15  engine-type        159 non-null    object 
 16  num-of-cylinders   159 non

For whatever reason, some numbers were written as words! converting to int...

In [54]:
%pip install word2number
from word2number import w2n

strangeColumns = ["num-of-doors", "num-of-cylinders"]
standardColumns = list(set(columns) - set(strangeColumns))

strangeData = dataDropped[strangeColumns]
standardData = dataDropped[standardColumns]

strangeData = strangeData.map(lambda a : w2n.word_to_num(str(a)))

data = pd.concat([standardData, strangeData], axis=1)
data.info()
print(data)

Note: you may need to restart the kernel to use updated packages.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   stroke             159 non-null    float64
 1   horsepower         159 non-null    float64
 2   bore               159 non-null    float64
 3   normalized-losses  159 non-null    float64
 4   height             159 non-null    float64
 5   curb-weight        159 non-null    int64  
 6   width              159 non-null    float64
 7   engine-size        159 non-null    int64  
 8   symboling          159 non-null    int64  
 9   drive-wheels       159 non-null    object 
 10  engine-location    159 non-null    object 
 11  highway-mpg        159 non-null    int64  
 12  compression-ratio  159 non-null    float64
 13  fuel-system        159 non-null    object 
 14  body-style         159 non-null    object 
 15  city-mpg


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Strings found! Using one-hot encoding to change data to numeric

In [55]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

strangeColumns = ["engine-location", "body-style", "drive-wheels", "engine-type", "fuel-type", "make", "fuel-system", "aspiration"]
standardColumns = list(set(columns) - set(strangeColumns))

strangeData = pd.DataFrame()
standardData = data[standardColumns]

for column in strangeColumns:
    local_cat = data[[column]]  
    local_cat.sample(10)

    encoder = OneHotEncoder()
    local_cat = encoder.fit_transform(local_cat).toarray()
    local_data = pd.DataFrame(local_cat, columns=encoder.get_feature_names_out([column]))
    strangeData = pd.concat([strangeData, local_data], axis=1)
    print(local_data)
dataCleaned = pd.concat([strangeData, standardData], axis=1)
print("#" * 100)
print(data)
#data.info()

     engine-location_front
0                      1.0
1                      1.0
2                      1.0
3                      1.0
4                      1.0
..                     ...
154                    1.0
155                    1.0
156                    1.0
157                    1.0
158                    1.0

[159 rows x 1 columns]
     body-style_convertible  body-style_hardtop  body-style_hatchback  \
0                       0.0                 0.0                   0.0   
1                       0.0                 0.0                   0.0   
2                       0.0                 0.0                   0.0   
3                       0.0                 0.0                   0.0   
4                       0.0                 0.0                   0.0   
..                      ...                 ...                   ...   
154                     0.0                 0.0                   0.0   
155                     0.0                 0.0                   0.

Unscaled values found! Using Standard scaler to scale

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

newColumns = list(dataCleaned.columns.values)

target = ["price", "peak-rpm", "horsepower"]
X = dataCleaned[list(set(newColumns) - set(target))]
y = dataCleaned[target]

X_train, X_valtest, y_train, y_valtest = train_test_split(X, y, test_size=0.4, random_state=None)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=None)

train_set_scaled = scaler.fit_transform(X_train)
X_train_scaled_df = pd.DataFrame(train_set_scaled, columns=X_train.columns)
X_train_scaled_df.head()

,make_jaguar,stroke,engine-type_l,fuel-type_gas,fuel-system_spdi,normalized-losses,height,make_dodge,engine-type_dohc,engine-location_front,...,make_nissan,wheel-base,fuel-type_diesel,num-of-doors,make_saab,length,drive-wheels_rwd,aspiration_std,engine-type_ohcv,fuel-system_1bbl
0,0.0,-2.396018,-0.235702,0.361873,-0.146647,-1.068007,0.477237,-0.209657,-0.180579,0.0,...,-0.398167,-0.201019,-0.361873,0.890198,-0.103142,0.175177,-0.662868,-2.405351,-0.235702,-0.259645
1,0.0,-0.351872,-0.235702,0.361873,-0.146647,-0.540000,0.115622,-0.209657,-0.180579,0.0,...,-0.398167,-0.960893,-0.361873,-1.123345,-0.103142,-1.154332,-0.662868,0.415740,-0.235702,-0.259645
2,0.0,0.890649,-0.235702,0.361873,6.819091,0.377064,-1.014426,-0.209657,-0.180579,0.0,...,-0.398167,-0.320999,-0.361873,0.890198,-0.103142,0.065148,-0.662868,0.415740,-0.235702,-0.259645
3,0.0,0.209267,-0.235702,0.361873,-0.146647,-0.567790,-0.155590,-0.209657,-0.180579,0.0,...,2.511512,-0.680940,-0.361873,0.890198,-0.103142,-0.136570,-0.662868,0.415740,-0.235702,-0.259645
4,0.0,1.050974,-0.235702,0.361873,-0.146647,0.293695,-0.381599,-0.209657,-0.180579,0.0,...,-0.398167,0.098931,-0.361873,-1.123345,-0.103142,0.413571,1.508596,0.415740,-0.235702,-0.259645


In [57]:
from sklearn.linear_model import *

model = Ridge(0.018)
model.fit(X_train, y_train)

print(model.score(X_test, y_test))
print(model.coef_)

print(model.predict(X_test) - y_test)

0.6779438902562852
[[ 0.00000000e+00  8.00421371e+01 -5.63675578e+02 -8.03215378e+02
  -2.75063600e+02  1.30606872e+01 -3.50928719e+02 -3.72854953e+03
   1.18658308e+03  0.00000000e+00  7.77692929e+03 -1.72089243e+02
   2.37421392e+03 -3.87588464e+03  2.79129901e+03 -3.06917715e+02
   0.00000000e+00 -7.01603769e+02  1.53007111e+03 -2.48086313e+02
   8.03215378e+02 -4.99893123e+02  4.98022693e+03  4.77667985e+03
  -6.77449211e+02 -2.93788950e+03 -3.22882772e+01 -1.57100228e+03
  -1.57374672e+03 -1.59460745e+03  2.69200456e+00  1.10485570e+03
  -1.69259326e+03 -3.12626927e+03 -1.48190025e+03 -1.46104110e+03
   4.14586021e+02  1.90502261e+01 -9.18455230e+01  2.10445834e+02
  -7.01603769e+02 -1.03643774e+03  0.00000000e+00  6.47567591e+02
  -2.39933654e+02  4.65880937e+02 -1.14939027e+03 -8.19357823e+02
   3.24574111e+02  8.03215378e+02  1.71801467e+02  1.60369526e+03
  -1.01076808e+02 -1.47674468e+02 -1.10485570e+03  3.26782577e+02
  -2.55929846e+02]
 [ 0.00000000e+00  3.01825080e+02 -4.6